analyze split_data method from data_formatter class(GenericDataFormatter)

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from tensorflow.compat.v1 import Session, ConfigProto
from tensorflow.python.eager.context import PhysicalDevice
from typing import Dict, List, Union, Generator
import os
from numpy import load

In [ ]:
import sys
sys.path.insert(0, '..')

In [ ]:
from data_formatters.base import GenericDataFormatter, InputTypes, DataTypes
from data_formatters.electricity import ElectricityFormatter

In [ ]:
from expt_settings.configs import ExperimentConfig
from libs.hyperparam_opt import HyperparamOptManager
from libs.tft_model import TemporalFusionTransformer
import libs.utils as utils

In [ ]:
 if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
gpu: List[PhysicalDevice] = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [ ]:
# Tensorflow setup
default_keras_session: Session = tf1.keras.backend.get_session()
tf_config: ConfigProto = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

In [ ]:
file_path: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\electricity\data\electricity\hourly_electricity.csv'

In [ ]:
raw_data: DataFrame = pd.read_csv(file_path, index_col=0)

In [ ]:
config = ExperimentConfig('electricity', r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs')

In [ ]:
formatter: ElectricityFormatter = config.make_data_formatter()

# SPLIT THE DATA

In [ ]:
train, valid, test = formatter.split_data(raw_data)

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
raw_data.columns

In [ ]:
raw_data.head()

In [ ]:
train.head()

looks like the data_formatter scaled "power_usage", "hour", "day_of_week", "hours_from_start"

In [ ]:
column_definitions = formatter.get_column_definition()

In [ ]:
column_definitions

In [ ]:
train_samples, valid_samples = formatter.get_num_samples_for_calibration()

In [ ]:
# Sets up default params
fixed_params: Dict = formatter.get_experiment_params()
params: Dict = formatter.get_default_model_params()
params["model_folder"]: str = os.path.join(config.model_folder, "fixed")
model_folder = os.path.join(config.model_folder, "fixed")

In [ ]:
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                   fixed_params, model_folder)

In [ ]:
model_folder: str = opt_manager.hyperparam_folder

# ANALYZE PARAMETERS

In [ ]:
fixed_params.keys()

In [ ]:
#column definition
fixed_params.get('column_definition')

In [ ]:
fixed_params.get('known_categorical_inputs')

# TRAINING

In [ ]:
print("*** Running calibration ***")
print("Params Selected:")
for k in params:
    print("{}: {}".format(k, params[k]))
    
best_loss = np.Inf
for _ in range(1):
    tf1.reset_default_graph()
    with tf.Graph().as_default(), tf1.Session(config=tf_config) as sess:
        tf1.keras.backend.set_session(sess)
        
        params: Dict = opt_manager.get_next_parameters()
        model: TemporalFusionTransformer = TemporalFusionTransformer(params, use_cudnn=False)

        if not os.path.exists(os.path.join(model.data_folder, 'data.npy')) and not model.training_data_cached():
            model.cache_batched_data(train, "train", num_samples=train_samples)
            model.cache_batched_data(valid, "valid", num_samples=valid_samples)
            
        sess.run(tf1.global_variables_initializer())
        model.fit() 
        
        val_loss: Series = model.evaluate()
            
        if val_loss < best_loss:
            opt_manager.update_score(params, val_loss, model)
            best_loss = val_loss
        tf1.keras.backend.set_session(default_keras_session)

In [ ]:
model_ckpt: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\saved_models\electricity\fixed\TemporalFusionTransformer.check'

In [ ]:
folder: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\electricity\data\electricity'
train_size: int = 450000

In [ ]:
def traindata_gen(folder: str, train_size: int , minibatch_size: int) -> Generator:
    data: memmap = load(os.path.join(folder, 'data.npy'), mmap_mode='r')
    print(type(data))
    labels: memmap = load(os.path.join(folder, 'labels.npy'), mmap_mode='r')
    active_flags: memmap = load(os.path.join(folder, 'active_flags.npy'), mmap_mode='r')
    train_size: Tuple = data.shape
    for i in range(train_size[0] // minibatch_size + 1):
        upper = min((i + 1) * minibatch_size, train_size[0])

        yield data[i * minibatch_size:upper], np.concatenate([labels[i * minibatch_size:upper],
                                                                           labels[i * minibatch_size:upper],
                                                                           labels[i * minibatch_size:upper]], axis=-1), \
                      active_flags[i * minibatch_size:upper]

In [ ]:
for data, labels, active_flags in traindata_gen(folder, train_size, 128):
    print(data.shape, labels.shape, active_flags.shape)